In [29]:
import pandas as pd
import plotly.express as px

In [30]:
taux = 0.0375
emprunt = 200000
duree = 25
taux_assurance = 0.0035
date_debut_pret = "2024-05-01"

In [46]:
import pandas as pd

def vpm(taux, duree, emprunt):
    '''
    Calcule la mensualité d'un emprunt
    '''
    taux_mensuel = taux / 12
    duree_mois = duree * 12
    vpm = (emprunt * taux_mensuel) / (1 - (1 + taux_mensuel)**(-duree_mois))
    vpm = round(vpm, 3)
    return vpm


def interet(taux, periode, duree, emprunt):
    '''
    Calcule l'intérêt d'une période donnée
    '''
    taux_mensuel = taux / 12
    duree_mois = duree * 12
    vpm = (emprunt * taux_mensuel) / (1 - (1 + taux_mensuel)**(-duree_mois))
    capital_restant = emprunt
    for i in range(1, periode):
        interet_mois = capital_restant * taux_mensuel
        capital_restant -= (vpm - interet_mois)
    interet_periode = capital_restant * taux_mensuel
    interet_periode = round(interet_periode, 3)
    return interet_periode


def create_table(taux, emprunt, duree, taux_assurance, date_debut_pret):
    mensualite_hors_assurance = vpm(taux, duree, emprunt)
    
    data = pd.DataFrame(range(1, duree*12+1), columns=['numero_mois'])
    data['annee'] = (data['numero_mois'] - 1) // 12 + 1
    
    day_date_debut_pret = pd.to_datetime(date_debut_pret).day
    if day_date_debut_pret == 1:
        data['date_remboursement'] = pd.date_range(date_debut_pret, periods=duree*12, freq='MS')
    else:
        date_debut_pret_ajusted_month = pd.to_datetime(date_debut_pret) + pd.DateOffset(months=-1)
        data['date_remboursement'] = pd.date_range(date_debut_pret_ajusted_month, periods=duree*12, freq='MS') + pd.DateOffset(days = day_date_debut_pret-1)
        
    data['date_remboursement'] = data['date_remboursement'].dt.strftime('%Y-%m-%d')
        
    data['mensualite_hors_assurance'] = mensualite_hors_assurance
    data['mensualite_interet'] = data.apply(lambda x: interet(taux, x['numero_mois'], duree, emprunt), axis=1)
    data['mensualite_capital'] = data['mensualite_hors_assurance'] - data['mensualite_interet']
    data['reste_a_rembourser'] = emprunt - data['mensualite_capital'].cumsum()
    data['mensualite_assurance'] = round(data['reste_a_rembourser'] * taux_assurance / 12, 3)
    data['mensualite_globale'] = mensualite_hors_assurance + data['mensualite_assurance']

    return data


data = create_table(taux = 0.035, emprunt = 250000, duree = 20, taux_assurance = 0.002, date_debut_pret = '2024-05-01')

data


,numero_mois,annee,date_remboursement,mensualite_hors_assurance,mensualite_interet,mensualite_capital,reste_a_rembourser,mensualite_assurance,mensualite_globale
0,1,1,2024-05-01,1449.899,729.167,720.732,249279.268,41.547,1491.446
1,2,1,2024-06-01,1449.899,727.065,722.834,248556.434,41.426,1491.325
2,3,1,2024-07-01,1449.899,724.956,724.943,247831.491,41.305,1491.204
3,4,1,2024-08-01,1449.899,722.842,727.057,247104.434,41.184,1491.083
4,5,1,2024-09-01,1449.899,720.721,729.178,246375.256,41.063,1490.962
...,...,...,...,...,...,...,...,...,...
235,236,20,2043-12-01,1449.899,20.961,1428.938,5757.615,0.960,1450.859
236,237,20,2044-01-01,1449.899,16.793,1433.106,4324.509,0.721,1450.620
237,238,20,2044-02-01,1449.899,12.613,1437.286,2887.223,0.481,1450.380
238,239,20,2044-03-01,1449.899,8.421,1441.478,1445.745,0.241,1450.140


In [32]:
fig = px.line(data, x='date_remboursement', y='mensualite_hors_assurance', title='Evolution de la réparition des mensualités')
fig.add_scatter(x=data['date_remboursement'], y=data['mensualite_capital'], mode='lines', name='Capital')
fig.add_scatter(x=data['date_remboursement'], y=data['mensualite_interet'], mode='lines', name='Intérets')

In [33]:
def capacite_emprunt(endettement, salaire, assurance, taux, duree, emprunt):
    '''
    Calcule la capacité d'emprunt en fonction du taux d'endettement
    '''
    endettement = 0.35
    salaire = 5000 # net avant impôt
    assurance = 0.002
    taux = 0.03
    duree = 20
    emprunt = 250000
    mensualite = vpm(taux, duree, emprunt) + (assurance * 250000 / 12)
    mensualite_max = salaire * endettement
    capacite_emprunt = mensualite_max / (vpm(taux / 12, duree * 12, -1) + assurance * emprunt / 12)
    return capacite_emprunt

capacite_emprunt(0.35, 5000, 0.002, 0.03, 20, 250000)

42.0

In [34]:
5000*0.35

1750.0

In [35]:
1750*12*20

420000